In [2]:
import cv2
import numpy as np
import os
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

def generate_face(results):

    if not results.face_landmarks:
        return np.zeros(160)

    face = np.array([[res.x, res.y] for res in results.face_landmarks.landmark]) if results.face_landmarks else np.zeros(468*3)
    x_min, x_max, y_min, y_max = np.min(face[:,0]), np.max(face[:,0]), np.min(face[:,1]), np.max(face[:,1])
    closest_points = []

    for y in np.linspace(y_min,y_max,20):
        closest_points.append(np.argmin(np.abs(face[:,0] - x_min) + np.abs(face[:,1] - y)))
        closest_points.append(np.argmin(np.abs(face[:,0] - x_max) + np.abs(face[:,1] - y)))

    for x in np.linspace(x_min,x_max,20):
        closest_points.append(np.argmin(np.abs(face[:,0] - x) + np.abs(face[:,1] - y_max)))
        closest_points.append(np.argmin(np.abs(face[:,0] - x) + np.abs(face[:,1] - y_min)))

    return np.array([[face[i,0], face[i,1]] for i in closest_points]).flatten()


def extract_keypoints(results):
    lh = np.array([[res.x, res.y] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*2)
    rh = np.array([[res.x, res.y] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*2)
    return np.concatenate([generate_face(results), lh, rh])

In [4]:
BACKUP_PATH = os.path.join("backup")
DATA_PATH = os.path.join('Data')
actions = np.array(['hello', 'world', 'from', 'germany'])
no_sequences = 200
sequence_length = 20

In [4]:
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

for action in actions:
    if not os.path.exists(os.path.join(DATA_PATH, action)):
        os.makedirs(os.path.join(DATA_PATH, action))
    if not os.path.exists(os.path.join(BACKUP_PATH, action)):
        os.makedirs(os.path.join(BACKUP_PATH, action))

In [5]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        while not cv2.waitKey(10) & 0xFF == ord('c'):
            ret, frame = cap.read()
            image, _ = mediapipe_detection(frame, holistic)

            cv2.putText(image, '{}, press c to start'.format(action), (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', image)
        for sequence in range(no_sequences):
            seq = []
            # Loop through video length aka sequence length
            if sequence != 0 and sequence%40 == 0:
                while not cv2.waitKey(10) & 0xFF == ord('c'):
                    ret, frame = cap.read()
                    image, _ = mediapipe_detection(frame, holistic)

                    cv2.putText(image, 'Take a break. press c to continue', (120,200),
                                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
            while True:

                if len(seq) >= sequence_length:
                    break

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # NEW Apply wait logic
                if len(seq) == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)

                # NEW Export keypoints
                if results.right_hand_landmarks or results.right_hand_landmarks:
                    keypoints = extract_keypoints(results)
                    seq.append(keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            npy_path = os.path.join(BACKUP_PATH, action, str(sequence))
            np.save(npy_path, np.array(seq))

    cap.release()
    cv2.destroyAllWindows()

# Only keep neccessary args

In [6]:
break
data_from_to = {
    "face": (0, 160),
    "lh": (160, 202),
    "rh": (202, 244)
}


def remove_unnec(to_rem: list):
    to_rem.sort()
    print(to_rem)
    for action in actions:
        n_seq = len(os.listdir(os.path.join(BACKUP_PATH, action)))
        for seq in range(n_seq):
            s = []
            data = np.load(os.path.join(BACKUP_PATH, action, f"{seq}.npy"))
            for frame in data:
                am_rem = 0
                for r in to_rem:
                    if r in data_from_to:
                        frame = np.delete(frame, [range(data_from_to[r][0]-am_rem, data_from_to[r][1]-am_rem)])
                        am_rem += data_from_to[r][1]-data_from_to[r][0]+1
                s.append(frame)
            npy_path = os.path.join(DATA_PATH, action, str(seq))
            np.save(npy_path, np.array(s))



remove_unnec(["face"])

_data = np.load(os.path.join(DATA_PATH, "learn", "0.npy"))

['face']
84


In [9]:
break
# Probably temp
folders = ["hello", "world", "from", "germany"]

for folder in folders:
    print(folder)
    for file in os.listdir(os.path.join(BACKUP_PATH, folder)):
        num = int(str(file).split(".")[0]) + 200
        old = os.path.join(BACKUP_PATH, folder, str(file))
        new = os.path.join(BACKUP_PATH, folder, f"{num}.npy")
        os.rename(old, new)



hello
world
from
germany
